In [5]:
with open ("wizard_of_oz.txt","r",encoding="utf-8") as f:
    text = f.read()
print(len(text))
print(text[:200])

230464
Dorothy and the Wizard in Oz


  A Faithful Record of Their Amazing Adventures
    in an Underground World; and How with the
     Aid of Their Friends Zeb Hugson, Eureka
       the Kitten, and Jim the


In [10]:
chars = sorted(set(text))
print(chars)
print(len(chars))

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
75


In [13]:
#Char level encoder - decoder (tokenizer-Char level)

string_to_int  = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}

encoder = lambda s: [string_to_int[c] for c in s]
decoder = lambda l: "".join([int_to_string[i] for i in l])


#test drivers
hello_encode = encoder("hello")
print(hello_encode)
decode_hello = decoder(hello_encode)
print(decode_hello)

[56, 53, 60, 60, 63]
hello


In [15]:
# Using pytorch

import torch

In [16]:
data = torch.tensor(encoder(text),dtype=torch.long)

data[:100]

tensor([27, 63, 66, 63, 68, 56, 73,  1, 49, 62, 52,  1, 68, 56, 53,  1, 46, 57,
        74, 49, 66, 52,  1, 57, 62,  1, 38, 74,  0,  0,  0,  1,  1, 24,  1, 29,
        49, 57, 68, 56, 54, 69, 60,  1, 41, 53, 51, 63, 66, 52,  1, 63, 54,  1,
        43, 56, 53, 57, 66,  1, 24, 61, 49, 74, 57, 62, 55,  1, 24, 52, 70, 53,
        62, 68, 69, 66, 53, 67,  0,  1,  1,  1,  1, 57, 62,  1, 49, 62,  1, 44,
        62, 52, 53, 66, 55, 66, 63, 69, 62, 52])

In [17]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]


# How this model is gonna work:

start -> h <br>
h -> e <br>
e -> l <br>
l -> l <br>
l -> o <br>

For now it will just predict the next char.

<br>

here we will make the dataset by block size. if I consider block size = 5 and offset = 1 then the fist col will be : <br>
n = 0 to n = 4 --> Feature var (X) <br>
n = 5 --> target var (y)<br>
next one will be<br>
n = 1 to n = 5 --> Feature var (X)<br>
n = 6 --> target var <br>
and so onn...<br>

i = 0;<br>
X = data[i:block_size+i], y = data[i+block_size+1] where i> 0;<br>
i+=1<br>

In [19]:
#creating the dataset with above principles:

block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("When Input is",context,"target is",target)

When Input is tensor([27]) target is tensor(63)
When Input is tensor([27, 63]) target is tensor(66)
When Input is tensor([27, 63, 66]) target is tensor(63)
When Input is tensor([27, 63, 66, 63]) target is tensor(68)
When Input is tensor([27, 63, 66, 63, 68]) target is tensor(56)
When Input is tensor([27, 63, 66, 63, 68, 56]) target is tensor(73)
When Input is tensor([27, 63, 66, 63, 68, 56, 73]) target is tensor(1)
When Input is tensor([27, 63, 66, 63, 68, 56, 73,  1]) target is tensor(49)


In [20]:
# Inroducing GPU accleration

device = 'mps' if torch.backends.mps.is_available() else 'cpu'
print(device)

mps


In [ ]:
block_size = 8
batch_size = 4